In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preprocessing

## Installing necessary libraries

In [ ]:
!pip install -q pybangla
!pip install -q git+https://github.com/csebuetnlp/normalizer
!pip install -q bnbphoneticparser
!pip install -q bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 70.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.4 MB/s eta 0:00:0

In [ ]:
import pandas as pd
import pybangla
import re

In [ ]:
!cp "/content/drive/MyDrive/Dataset-Sentiment Analysis (Task-ML Engineer).xlsx" "."

In [ ]:
df = pd.read_excel('/content/Dataset-Sentiment Analysis (Task-ML Engineer).xlsx')

In [ ]:
df.head()

,conversation_text,sentiment
0,company_x এজেন্ট থেকে রং নাম্বারে টাকা গেলে ফে...,neutral
1,company_x গ্রাহকদের জন্য খুবই দুঃখ জনক একটা খব...,negative
2,company_x গ্রাহকদের জন্য খুবই দুঃখ জনক একটা খব...,negative
3,bank_y ব্যাংক যখন ঘোষনা দিয়েছে যে কার্ড দিয়ে ক...,negative
4,bank_y ব্যাংক যখন ঘোষনা দিয়েছে যে কার্ড দিয়ে ক...,negative


## EDA and Preprocessing

In [ ]:
df.sentiment.value_counts()

,count
sentiment,
neutral,45
negative,42
positive,12


In [ ]:
df['word_count'] = df['conversation_text'].apply(lambda x: len(x.split()))

average_word_count = df['word_count'].mean()
minimum_word_count = df['word_count'].min()
maximum_word_count = df['word_count'].max()

print(f'Average word count: {average_word_count}')
print(f'Minimum word count: {minimum_word_count}')
print(f'Maximum word count: {maximum_word_count}')

Average word count: 43.44444444444444
Minimum word count: 5
Maximum word count: 118


In [ ]:
nrml = pybangla.Normalizer()

In [ ]:
df.conversation_text = df.conversation_text.apply(nrml.text_normalizer)
df.conversation_text = df.conversation_text.apply(nrml.remove_emoji)

In [ ]:
df.conversation_text[3]

'bank_y ব্যাংক যখন ঘোষনা দিয়েছে যে কার্ড দিয়ে ক্যাশ উত্তোলন বন্ধ তেমনি ভাবে আবার যে কোন সময় company_xের ক্যাশআউট বন্ধ করে দিতে পারে। কারন company_xের মূল মালিক bank_y ব্যাংক See Translation'

In [ ]:
from bnbphoneticparser import BanglishToBengali


banglish2bengali = BanglishToBengali()
operators = ['=', '+', '-', '*', '/', '%', '<', '>', '×', '÷']
punctuation_marks = ['।', ',', ';', ':', '?', '!', "'", '.', '"', '-', '[', ']', '{', '}', '(', ')', '–', '—', '―', '~']

chars_to_remove = operators + punctuation_marks

pattern = f"[{''.join(map(re.escape, chars_to_remove))}]"

def remove_chars(text):
    text = re.sub(pattern, '', text)
    text = re.sub('See Translation', ' ', text)
    text = re.sub('_', ' ', text)
    text = re.sub('company', 'কোম্পানি', text)
    text = re.sub('The Business Standard', 'বিজনেস স্ট্যান্ডার্ড', text)
    return banglish2bengali.parse(text.strip())

df['sentence'] = df['conversation_text'].apply(remove_chars)

In [ ]:
df.sentence[3]

'বাঙ্ক য় ব্যাংক যখন ঘোষনা দিয়েছে যে কার্ড দিয়ে ক্যাশ উত্তোলন বন্ধ তেমনি ভাবে আবার যে কোন সময় কোম্পানি এক্সের ক্যাশআউট বন্ধ করে দিতে পারে কারন কোম্পানি এক্সের মূল মালিক বাঙ্ক য় ব্যাংক'

In [ ]:
df.to_csv('x.csv') # optional step

In [ ]:
from normalizer import normalize

def buet_normal(input_text):
  return  normalize(
      input_text,
      unicode_norm="NFKC",
      punct_replacement=None,
      url_replacement=None,
      emoji_replacement=None,
      apply_unicode_norm_last=True
  )

df.sentence = df.sentence.apply(buet_normal)

In [ ]:
df.sentence[3]

'বাঙ্ক য় ব্যাংক যখন ঘোষনা দিয়েছে যে কার্ড দিয়ে ক্যাশ উত্তোলন বন্ধ তেমনি ভাবে আবার যে কোন সময় কোম্পানি এক্সের ক্যাশআউট বন্ধ করে দিতে পারে কারন কোম্পানি এক্সের মূল মালিক বাঙ্ক য় ব্যাংক'

In [ ]:
df['word_count'] = df['sentence'].apply(lambda x: len(x.split()))

average_word_count = df['word_count'].mean()
minimum_word_count = df['word_count'].min()
maximum_word_count = df['word_count'].max()

print(f'Average word count: {average_word_count}')
print(f'Minimum word count: {minimum_word_count}')
print(f'Maximum word count: {maximum_word_count}')

Average word count: 46.474747474747474
Minimum word count: 6
Maximum word count: 153


# Training and Testing Model

## Deep Learning Model (Stacked LSTM)

In [ ]:
!wget https://huggingface.co/sagorsarker/bangla-glove-vectors/resolve/main/bn_glove.39M.100d.zip

--2024-09-11 15:36:55--  https://huggingface.co/sagorsarker/bangla-glove-vectors/resolve/main/bn_glove.39M.100d.zip
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.17, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/aa/54/aa548a31e1afcb96a938d527a7a3914d612289d4eec18ddd467509e7f2f37969/81c366cca4b903e0ccb3f1de51fa3bb227a33920e08c2cb302e2fe1e3753cd5a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27bn_glove.39M.100d.zip%3B+filename%3D%22bn_glove.39M.100d.zip%22%3B&response-content-type=application%2Fzip&Expires=1726328215&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNjMyODIxNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9hYS81NC9hYTU0OGEzMWUxYWZjYjk2YTkzOGQ1MjdhN2EzOTE0ZDYxMjI4OWQ0ZWVjMThkZGQ0Njc1MDllN2YyZjM3OTY5LzgxYzM2NmNjYTRiOTAzZTBjY

In [ ]:
!unzip -q '/content/bn_glove.39M.100d.zip'

In [ ]:
import numpy as np
embeddings_dict = {}
with open("/content/bn_glove.39M.100d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from bnlp import NLTKTokenizer

bnltk = NLTKTokenizer()
df['tokenized'] = df.sentence.map(bnltk.word_tokenize)

In [ ]:
def word_vectors(sentence):
    vectors = []
    for word in sentence:
      if word in embeddings_dict.keys():
        vectors.append(embeddings_dict[word])
    return vectors

In [ ]:
sentence_vectors = [word_vectors(sentence) for sentence in df['tokenized']]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = 150
padded_sequences = pad_sequences(sentence_vectors,
                                 maxlen=max_sequence_length,
                                 padding='post',
                                 truncating='post',
                                 dtype='float32')

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
df.sentiment.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [ ]:
categories = {
    'neutral': 0,
    'negative': 1,
    'positive': 2
}

def categorical(sentiment):
  return categories[sentiment]

df['label'] = df.sentiment.apply(categorical)

In [ ]:
X = np.asarray(padded_sequences).astype('float32')
y = np.array(df.label.to_list())

### Spliting Data

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.3,  # 40% of the data goes to the temporary set
    random_state=37,
    shuffle=True
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,  # 50% of the temporary set goes to the test set
    random_state=37,
    shuffle=True
)

In [ ]:
len(X_train), len(X_val), len(X_test)

(69, 15, 15)

In [ ]:
print(X_train.shape)

(69, 150, 100)


### Training Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Activation, Dropout, Flatten, Embedding
from keras.layers import Conv1D,MaxPooling1D,GlobalAveragePooling1D, Bidirectional, LSTM, GRU
from keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, Nadam, RMSprop
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model


embedding_dim = 100
number_of_classes = 3


input_layer = Input(shape=(max_sequence_length, embedding_dim))

lstm_layer_1 = Bidirectional(LSTM(32, return_sequences=True))(input_layer)
lstm_layer_2 = Bidirectional(LSTM(16, return_sequences=True))(lstm_layer_1)
lstm_layer_3 = Bidirectional(LSTM(8))(lstm_layer_2)

output_layer = Dense(number_of_classes, activation='softmax')(lstm_layer_3)

stacked_lstm_model = Model(inputs=input_layer, outputs=output_layer)

stacked_lstm_model.compile(optimizer = 'adam',
                      loss = 'sparse_categorical_crossentropy',
                      metrics = ['accuracy'])

stacked_lstm_model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 150, 100)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_12 (Bidirectional)     │ (None, 150, 64)             │          34,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ (None, 150, 32)             │          10,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_14 (Bidirectional)     │ (None, 16)                  │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,091 (183.95 KB)

 Trainable params: 47,091 (183.95 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
  # Saved the Best Model
filepath = "stacked_bilstm.keras"
checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                             monitor='val_loss',
                                             verbose=2,
                                             save_best_only=True,
                                             save_weights_only=False,
                                             mode='max')
#   # callback list
callback_list = [acc_callback, checkpoint]

history = stacked_lstm_model.fit(
                              X_train,
                              y_train,
                              epochs = 10,
                              batch_size = 32,
                              verbose = 1,
                              validation_data =(X_val, y_val),
                              callbacks = [callback_list]
                              )

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.4522 - loss: 1.0441
Epoch 1: val_loss improved from -inf to 1.08944, saving model to stacked_bilstm.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 861ms/step - accuracy: 0.4479 - loss: 1.0442 - val_accuracy: 0.4000 - val_loss: 1.0894
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.4783 - loss: 0.9663
Epoch 2: val_loss improved from 1.08944 to 1.09718, saving model to stacked_bilstm.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step - accuracy: 0.4674 - loss: 0.9696 - val_accuracy: 0.4667 - val_loss: 1.0972
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.6854 - loss: 0.9185
Epoch 3: val_loss improved from 1.09718 to 1.11254, saving model to stacked_bilstm.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step - accuracy: 0.6843 - loss: 0.9242 - val_accuracy: 0.4000 - val_loss: 1.1125
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.6653 - loss: 0.8870
Epoch 4: val_loss improved from 1.11254 to 1.13352, sa

### Testing Model

In [ ]:
y_pred_test = stacked_lstm_model.predict(X_test)
y_pred_test = np.argmax(y_pred_test, axis=-1)
y_pred_test[:5]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([1, 1, 0, 1, 0])

In [ ]:
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.75      0.30      0.43        10
           1       0.27      0.75      0.40         4
           2       0.00      0.00      0.00         1

    accuracy                           0.40        15
   macro avg       0.34      0.35      0.28        15
weighted avg       0.57      0.40      0.39        15



In [ ]:
confusion_matrix(y_test,y_pred_test)

array([[3, 7, 0],
       [1, 3, 0],
       [0, 1, 0]])

# Training & Testing BanglaBERT

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install git+https://github.com/csebuetnlp/normalizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is 

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-_26p3gw6
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-_26p3gw6
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
  Using cached emoji-1.4.2-py3-none-any.whl
  Using cached ftfy-6.0.3-py3-none-any.whl
  Attempting uninstall: emoji
    Found existing installation: emoji 1.7.0
    Uninstalling emoji-1.7.0:
      Successfully uninstalled emoji-1.7.0
  Attempting uninstall: ftfy
    Found existing installation: ftfy 6.2.0
    Uninstalling ftfy-6.2.0:
      Successfully uninstalled ftfy-6.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bnlp-toolkit 4.0.3 requires emoji==1.7.0, but you have emoji 1.4.2 which is incompatible.
bnlp-t

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [ ]:
# We'll only use the normalized text,
# BanglaBERT can all three forms Bangla, English, Banglish
# So no worries
categories = {
    'neutral': 0,
    'negative': 1,
    'positive': 2
}

def categorical(sentiment):
  return categories[sentiment]

df['label'] = df.sentiment.apply(categorical)

from sklearn.model_selection import train_test_split

df_train, df_temp = train_test_split(df, test_size=0.3, random_state=37)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=37)

df_train.to_csv('train.csv', index = False)
df_val.to_csv('val.csv', index = False)
df_test.to_csv('test.csv', index = False)

In [ ]:
train_split = '/content/train.csv'
val_split = '/content/val.csv'
test_split = '/content/test.csv'

from datasets import load_dataset
train_dataset = load_dataset("csv", data_files=train_split)
val_dataset = load_dataset("csv", data_files=val_split)
test_dataset = load_dataset("csv", data_files=test_split)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset['train'][0]

{'conversation_text': 'company_xের নতুন ফিচার নিয়ে একটু চিন্তিত আছি। company_xের নতুন ফিচার গ্রাহকদের থেকে ফেস রিকগনিশন এবং ফিঙ্গার প্রিন্ট নিবে এতে আমাদের পার্সোনাল ডাটা তারা নিয়ে নিচ্ছে। company_x এভাবে পার্সোনাল ডাটা নিয়ে সেভ রাখতে পারবে তো? যদি company_xের সার্ভার সিস্টেমের উপর সাইবার অ্যাটাক হয় বা কোন প্রবলেম হয় তাহলে আমাদের পার্সোনাল ডাটা লিক হয়ে যাবে তখন এর দায়ভার কে নিবে। company_xের এভাবে ফেস রিকগনিশন এবং ফিঙ্গার প্রিন্ট ডাটা নেয়া কি আদৌ ঠিক? See Translation',
 'sentiment': 'negative',
 'word_count': 71,
 'label': 1}

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "csebuetnlp/banglabert"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["conversation_text"], truncation=True, max_length=600)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/69 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets['train'].column_names

['conversation_text',
 'sentiment',
 'word_count',
 'label',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [ ]:
tokenized_train_datasets = tokenized_datasets.remove_columns(["sentiment", "word_count", "conversation_text"])
tokenized_train_datasets.set_format("torch")
print(tokenized_train_datasets["train"].column_names)

tokenized_eval_datasets = tokenized_val_datasets.remove_columns(["sentiment", "word_count", "conversation_text"])
tokenized_eval_datasets.set_format("torch")
print(tokenized_eval_datasets["train"].column_names)

tokenized_test_datasets = tokenized_test_datasets.remove_columns(["sentiment", "word_count", "conversation_text"])
tokenized_test_datasets.set_format("torch")
print(tokenized_test_datasets["train"].column_names)

['label', 'input_ids', 'token_type_ids', 'attention_mask']
['label', 'input_ids', 'token_type_ids', 'attention_mask']
['label', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
import evaluate
import numpy as np
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="micro")

In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

training_args = TrainingArguments("Fine-tuned-BanglaBERT",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=10,
                                  learning_rate = 6e-5,
                                  weight_decay = 5e-7,
                                  overwrite_output_dir=True,
                                  remove_unused_columns=False,
                                  local_rank= 1,
                                  load_best_model_at_end=True,
                                  save_total_limit=2,
                                  save_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  optim='adafactor'
                                  )
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
for param in model.parameters(): param.data = param.data.contiguous()

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_datasets["train"],
    eval_dataset=tokenized_eval_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# let's 🚀
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.101315,0.333333
2,No log,1.084098,0.533333
3,No log,0.929929,0.600000
4,No log,0.848889,0.600000
5,No log,0.907888,0.600000
6,No log,0.966989,0.533333
7,No log,0.919867,0.666667
8,No log,0.917115,0.666667
9,No log,0.918117,0.666667
10,No log,0.939664,0.666667


TrainOutput(global_step=50, training_loss=0.4247418975830078, metrics={'train_runtime': 58.5484, 'train_samples_per_second': 11.785, 'train_steps_per_second': 0.854, 'total_flos': 58788376486812.0, 'train_loss': 0.4247418975830078, 'epoch': 10.0})

## Prediction

In [ ]:
pred = trainer.predict(tokenized_test_datasets['train'])
y = pred.predictions
y_pred = np.argmax(y, axis=-1)
y_pred[:5]

array([1, 0, 0, 0, 0])

In [ ]:
y_true = tokenized_test_datasets['train']['label']

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

print(classification_report(y_true,y_pred))
print(confusion_matrix(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.70      0.78        10
           1       0.50      0.75      0.60         4
           2       1.00      1.00      1.00         1

    accuracy                           0.73        15
   macro avg       0.79      0.82      0.79        15
weighted avg       0.78      0.73      0.75        15

[[7 3 0]
 [1 3 0]
 [0 0 1]]


In [ ]:
!cp -rf "/content/Fine-tuned-BanglaBERT" "/content/drive/MyDrive/Markopolo"

# Fine-tuning LLM

Followed below tutorial
- https://www.datacamp.com/tutorial/fine-tuning-llama-3-1

In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install -U peft
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_temp = train_test_split(df, test_size=0.3, random_state=37)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=37)

In [ ]:
# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
            Classify the text into neutral, negative, positive, and return the answer as the corresponding sentiment label.
            text: {data_point["conversation_text"]}
            label: {data_point["sentiment"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the text into neutral, negative, positive, and return the answer as the corresponding sentiment label.
            text: {data_point["conversation_text"]}
            label: """.strip()

In [ ]:
# Generate prompts for training and evaluation data
df_train.loc[:,'text'] = df_train.apply(generate_prompt, axis=1)
df_val.loc[:,'text'] = df_val.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = df_test.loc[:,'sentiment']
df_test = pd.DataFrame(df_test.apply(generate_test_prompt, axis=1), columns=["text"])

In [ ]:
df_train.sentiment.value_counts()

,count
sentiment,
negative,33
neutral,29
positive,7


In [ ]:
train_data = Dataset.from_pandas(df_train[["text"]])
eval_data = Dataset.from_pandas(df_val[["text"]])

In [ ]:
train_data[0]

{'text': 'Classify the text into neutral, negative, positive, and return the answer as the corresponding sentiment label.\n            text: company_xের নতুন ফিচার নিয়ে একটু চিন্তিত আছি। company_xের নতুন ফিচার গ্রাহকদের থেকে  ফেস রিকগনিশন এবং ফিঙ্গার প্রিন্ট নিবে এতে আমাদের পার্সোনাল ডাটা তারা নিয়ে নিচ্ছে। company_x এভাবে পার্সোনাল ডাটা নিয়ে সেভ রাখতে পারবে তো? যদি company_xের সার্ভার সিস্টেমের উপর সাইবার অ্যাটাক হয় বা কোন প্রবলেম হয়  তাহলে আমাদের পার্সোনাল ডাটা লিক হয়ে যাবে তখন এর দায়ভার কে নিবে। company_xের এভাবে ফেস রিকগনিশন এবং ফিঙ্গার প্রিন্ট ডাটা নেয়া কি আদৌ ঠিক?See Translation\n            label: negative',
 '__index_level_0__': 65}

In [ ]:
base_model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ['neutral', 'negative', 'positive']

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

y_pred = predict(df_test, model, tokenizer)

100%|██████████| 15/15 [00:35<00:00,  2.34s/it]


In [ ]:
def evaluate(y_true, y_pred):
    labels = ['neutral', 'negative', 'positive']
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

evaluate(y_true, y_pred)

Accuracy: 0.533
Accuracy for label neutral: 0.500
Accuracy for label negative: 0.500
Accuracy for label positive: 1.000

Classification Report:
              precision    recall  f1-score   support

     neutral       0.71      0.50      0.59        10
    negative       0.40      0.50      0.44         4
    positive       0.33      1.00      0.50         1

    accuracy                           0.53        15
   macro avg       0.48      0.67      0.51        15
weighted avg       0.61      0.53      0.54        15


Confusion Matrix:
[[5 3 2]
 [2 2 0]
 [0 0 1]]


In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['up_proj', 'q_proj', 'gate_proj', 'v_proj', 'k_proj', 'o_proj', 'down_proj']

In [ ]:
output_dir="llama-3.1-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=256,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:327: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you pass

Map:   0%|          | 0/69 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
2,2.027000,1.947302
4,1.474000,1.658621
6,1.291200,1.489977
8,1.191000,1.461776


TrainOutput(global_step=8, training_loss=1.511931225657463, metrics={'train_runtime': 156.2476, 'train_samples_per_second': 0.442, 'train_steps_per_second': 0.051, 'total_flos': 578352541802496.0, 'train_loss': 1.511931225657463, 'epoch': 0.927536231884058})

In [ ]:
y_pred = predict(df_test, model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 15/15 [00:40<00:00,  2.73s/it]

Accuracy: 0.733
Accuracy for label neutral: 0.700
Accuracy for label negative: 0.750
Accuracy for label positive: 1.000

Classification Report:
              precision    recall  f1-score   support

     neutral       0.88      0.70      0.78        10
    negative       0.50      0.75      0.60         4
    positive       1.00      1.00      1.00         1

    accuracy                           0.73        15
   macro avg       0.79      0.82      0.79        15
weighted avg       0.78      0.73      0.75        15


Confusion Matrix:
[[7 3 0]
 [1 3 0]
 [0 0 1]]


In [ ]:
!cp -rf "/content/llama-3.1-fine-tuned-model" "/content/drive/MyDrive/Markopolo"